In [ ]:
import numpy as np
import time
import json
import http.client, urllib.parse
import tweepy
import re
import pickle

In [ ]:
def get_api():
  consumer_key = "F1Sj8AeVD4Ia2F3BzgkaRl5eI"
  consumer_secret = "AA1WBhNbEHtnYbqlAuSKnVpJSbYfy7RZDlSQoqvouaSigVymPt"
  access_token = "1267293225435291649-SDx2Lh5hAX2gkkegsarWdLHjabi4AG"
  access_token_secret = "Kx5NRJ0LnvkRuaFR9aBu7L4MBLeoNr5mPp7blfiVsD6iO"

  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)

  #auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
  api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
  return api

In [ ]:
inputpath = "/content/drive/MyDrive/CS245/hey/"
outputpath = "/content/drive/MyDrive/CS245/after10/4/"

In [ ]:
with open (inputpath+'tweets_1.pickle', 'rb') as fp:
  status_list = pickle.load(fp)

In [ ]:
def geocode(x):
  if(str(x).lower()=="none"):
    return 0

  conn = http.client.HTTPConnection('api.positionstack.com')
  if (re.search(r'([^A-Za-z\s\,\.])+',x)) is not None:
    # removes locations that contain special characters
    return 0
  if (re.search(r'^(([A-Z]+([a-z\,\.])*)+(\s)*)+$',x)) is None:
    #removes locations not in title case
    return 0
  params = urllib.parse.urlencode({
      'access_key': '4cb88d6b908d2800c9abc0aee37042a3',
      'query': x
      })

  conn.request('GET', '/v1/forward?{}'.format(params))

  res = conn.getresponse()
  data = res.read()
  data = data.decode('utf-8')
  try:
    data2 = json.loads(data)
    temp = len(data2['data'])
    if (temp):
      total_conf = 0
      coordinates = [0,0]
      for i in range(temp):
        conf=data2['data'][i]['confidence']
        if(conf>=0.5):
          coordinates[0] += data2['data'][i]['longitude']*conf
          coordinates[1] += data2['data'][i]['latitude']*conf
          total_conf+=conf
      if(total_conf>0):
        coordinates[0] = coordinates[0]/total_conf
        coordinates[1] = coordinates[1]/total_conf
        return coordinates
    else:
      return 0
  except:
    return 0

In [ ]:
class runner:
  def __init__(self):
    self.current = 0
  def run(self, a, run_num, maximum, batch_size):
    self.current = a
    statuses = status_list[a:]
    print(a)
    total = maximum-a
    batch_size = batch_size
    batch_count = 0
    end = 0
    api = get_api()
    while (end<total):
      selected = []
      start = batch_count*batch_size
      end = min((batch_count+1)*batch_size, total)
      print(start, end)
      batch_count+=1
      if (batch_count%4==0):
        # refresh api
        #account = 1-account
        api = get_api()
      count = 0
      count_found = 0
      for status in statuses[start:end]:
        count+=1
        if (count%20 ==0):
          print(f"upto number {(batch_count-1)*batch_size+count}")
        user = status.user
        if (status.coordinates):
          other_tweets = api.user_timeline(user.id, count=200)
          number = len(other_tweets)
          dic = {}
          dic['user'] = user
          dic['status'] = status
          dic['tweets'] = other_tweets[0:use_num]
          dic['location'] = status
          dic['type'] = 1
          selected.append(dic)
          count_found+=1
          print("found type 1")
          continue
        elif (status.place):
          other_tweets = api.user_timeline(user.id, count=200)
          number = len(other_tweets)
          dic = {}
          dic['user'] = user
          dic['status'] = status
          dic['tweets'] = other_tweets[0:use_num]
          dic['location'] = status
          dic['type'] = 2
          selected.append(dic)
          count_found+=1
          print("found type 2")
          continue
        loc = geocode(user.location)
        if (loc):
          other_tweets = api.user_timeline(user.id, count=200)
          number = len(other_tweets)
          use_num = min(40,number)
          dic = {}
          dic['user'] = user
          dic['status'] = status
          dic['tweets'] = other_tweets
          dic['location'] = loc
          dic['type'] = 3
          selected.append(dic)
          count_found+=1
      print(f'## Batch Number {batch_count} ## \n count found = {count_found}/{count}')
      self.current = a+(batch_count)*batch_size #done before this run + done in previous batches + current count in batch
      print(f"current is {self.current}")
      with open(f'{outputpath}selected_{run_num}_{batch_count}.pickle', 'wb') as fp:
        pickle.dump(selected, fp)

In [ ]:
runs = 15
runner_obj = runner()
a = 38300
maximum = 40050
batch_size = 50
e = 0
while (a<maximum):
  try:
    print(f'###### Run number {runs} ######')
    runner_obj.run(a,runs, maximum, batch_size) 
    a = runner_obj.current
    print(runner_obj.current)
  except Exception as e:
    print(e)
    print("error found", runner_obj.current, runs)
    runs+=1
    a = runner_obj.current
    if (str(e) == "Not authorized."):
      a+=batch_size
      #time.sleep(240)
      print(a)
    time.sleep(60)
    #runner_obj.run(runner_obj.current, runs, account, maximum, batch_size)